# Follow Up Questions Generator

## Import Libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
import re 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from autocorrect import spell

## Loading Data

In [2]:
data=pd.read_csv("./followMLdata.csv")
data.head()

,Unnamed: 0,question,answer,followUp
0,0,Are you a self-motivator?,"Absolutely. For me, internal motivation works ...",Awesome. How would you spread motivation to ot...
1,1,What matters to you more - job satisfaction or...,"According to me, job satisfaction covers all -...",Are you ready to work in a company who offers ...
2,2,What matters to you more - job satisfaction or...,"According to me, job satisfaction covers all -...",If you are not able to adjust with your teamma...
3,3,"Have you worked with someone unprofessional, h...","During my B-Tech final semester internship, I ...",Do you think not showing the displeasure is th...
4,4,"Have you worked with someone unprofessional, h...","During my B-Tech final semester internship, I ...","If the same situation arises now, will you tak..."


In [3]:
que=data['question']
ans=data['answer']
flw=data['followUp']
stop_words = set(stopwords.words('english')) 
regex = re.compile('[%s]' % re.escape(string.punctuation))

## Spell Correction, Removing punctuation, Tokenizing and making word_to_vec dictionary

In [4]:
word_vector={}
for i in que.tolist():
    s=regex.sub(' ',i)
    words=word_tokenize(s)
    for word_tokens in words:
        w=word_tokens.lower()
        w=spell(w)
        word_vector[w]=[]

for i in ans.tolist():
    s=regex.sub(' ',i)
    words=word_tokenize(s)
    for word_tokens in words:
        w=word_tokens.lower()
        w=spell(w)
        word_vector[w]=[]

for i in flw.tolist():
    s=regex.sub(' ',i)
    words=word_tokenize(s)
    for word_tokens in words:
        w=word_tokens.lower()
        w=spell(w)
        word_vector[w]=[]
    
  
    #print(filtered_sentence)
 
    


## Determining maximum number sentences and words in each row in datasets

In [5]:
mx=0
for i in que:
    l=len(i.split('?'))
    if mx<l:
        mx=l
print("maxqsen ",mx)

mx=0
for i in ans:
    l=len(i.split('.'))
    if mx<l:
        mx=l
print("maxrsen ",mx)
mx=0
for i in que:
    l=i.split('?')
    for j in l:
        w=len(word_tokenize(j))
        if mx<w:
            mx=w
print("maxqword ",mx)
mx=0
for i in ans:
    l=i.split('.')
    for j in l:
        w=len(word_tokenize(j))
        if mx<w:
            mx=w
print("maxrword ",mx)
        
    
mx=0
for i in flw:
    l=len(word_tokenize(i))
    if mx<l:
        mx=l
print("maxfword ",mx)





maxqsen  4
maxrsen  21
maxqword  33
maxrword  79
maxfword  45


In [6]:
maxqsen=5
maxrsen=25
maxqword=50
maxrword=100
maxfword=50

emb=50

## Reading glove embedding and assigning to word dictonary

In [7]:

with open('./glove.6B/glove.6B.50d.txt','r',encoding="utf8") as file: 
    count=0
    cnt=0
    for index, line in enumerate(file): 
        #print(line)
        values = line.split()
        word=values[0]
        word_weights = np.asarray(values[1:], dtype=np.float32)
        if word in word_vector.keys():
            print("exist ",cnt)
            cnt=cnt+1
            word_vector[word]= word_weights
            
        #print(count," ",word)
        count=count+1
        #print(word)
        #print(word_weights)
        
    print(count," ",cnt)
        

exist  0
exist  949
exist  1779
exist  2619
exist  2620
exist  3349
exist  3350
400000   3509


## Finding words not available in glove embeddings

In [8]:
mispell=[]
cnt=0
for i in word_vector.keys():
    try:
        a=word_vector[i].shape
    except:
        print(i)
        mispell.append(i)
        cnt=cnt+1
print(cnt)

Lederer
algorithicm
concentratedly
unprossional
batchmates
Bangalore
Benz
youhave
weighage
IOT
Hackathorn
Hyderabad
schoolsindia
definedly
obsessivecompulsive
thuses
unorder
Kalyan
AIEEE
GitHub
intellective
dubaara
comfortableness
Bollywood
JavaScript
localstorage
calculative
incase
IP
direr
Alduino
APJ
FIFA
Chennai
usedly
Uno
Sudoku
sahyog
sagittarian
iiitb
TCS
reproval
Stallone
Woking
schwarztneger
persued
uncharge
linkedit
iitjee
retrospected
chitchatting
Microsoft
Abdul
YouTube
jokul
Mercedes
remotivation
youve
throwball
indited
criterial
MTech
Ve
63


## Preparing inputs and labels for training

In [9]:
# 4,21,33,79,45
print("shapes")
qs=[]
for i in que.tolist():
    temp=i.split('?')
    qn=[]
    for j in temp:
        s=regex.sub(' ',j)
        words=word_tokenize(s)
        qw=[]
        for word_tokens in words:
            w=word_tokens.lower()
            w=spell(w)

            if w in mispell:
                qw.append(word_vector['a'].tolist())
            else:
                qw.append(word_vector[w].tolist())
            
        b=maxqword-len(words)
        for t in range(b):
            qw.append(np.zeros((emb,)).tolist())
        
        qn.append(qw)
        
    b=maxqsen-len(temp) 
    for t in range(b):
        qw=[]
        for tt in range(maxqword):
            qw.append(np.zeros((emb,)).tolist())
        qn.append(qw)
        
            
        
    qs.append(qn)

qs=np.array(qs)
print("questions ",qs.shape)
    
# 4,21,33,79,45

rs=[]
rsqlen=[]
for i in ans.tolist():
    temp=i.split('.')
    rn=[]
    rsn=[]
    for j in temp:
        s=regex.sub(' ',j)
        words=word_tokenize(s)
        rw=[]
        for word_tokens in words:
            w=word_tokens.lower()
            w=spell(w)

            if w in mispell:
                rw.append(word_vector['a'].tolist())
            else:
                rw.append(word_vector[w].tolist())
            
        b=maxrword-len(words)
        for t in range(b):
            rw.append(np.zeros((emb,)).tolist())
        
        rn.append(rw)
        rsn.append(b)
        
    b=maxrsen-len(temp) 
    for t in range(b):
        rw=[]
        for tt in range(maxrword):
            rw.append(np.zeros((emb,)).tolist())
        rn.append(rw)
        rsn.append(0)
            
    rsqlen.append(rsn)   
    rs.append(rn)

rs=np.array(rs)
rsqlen=np.array(rsqlen)
print("responses ",rs.shape)
print("responses length ",rsqlen.shape)

       
fs=[]
fsqlen=[]

for i in flw.tolist():
    
    s=regex.sub(' ',i)
    words=word_tokenize(s)
    fw=[]
    for word_tokens in words:
        w=word_tokens.lower()
        w=spell(w)

        if w in mispell:
            fw.append(word_vector['a'].tolist())
        else:
            fw.append(word_vector[w].tolist())

    b=maxfword-len(words)
    for t in range(b):
        fw.append(np.zeros((emb,)).tolist())
   
    fs.append(fw)
    fsqlen.append(b)

fs=np.array(fs)
fsqlen=np.array(fsqlen)
print("followup questions ",fs.shape)
print("followup questions length ",fsqlen.shape)




shapes
questions  (1086, 5, 50, 50)
responses  (1086, 25, 100, 50)
responses length  (1086, 25)
followup questions  (1086, 50, 50)
followup questions length  (1086,)


## Making tensorflow computation graph

### CNN and Topk values in vector are use for making sentence representation

In [10]:
import tensorflow as tf
import numpy as np
batch_size=1086
tf.reset_default_graph()


g = tf.Graph()
with g.as_default():
    dtypei=tf.int32
    dtypef=tf.float32
    src_seq_length=tf.placeholder(dtypei,[None,maxrsen])
    trg_seq_length=tf.placeholder(dtypei,[None,])

    ns=20
    a=5
    dtypef=tf.float32
    dtypei=tf.int32
    inputsq=tf.placeholder(dtypef,[None,maxqsen,maxqword,emb])
    inputsr=tf.placeholder(dtypef,[None,maxrsen,maxrword,emb])
    
    with tf.variable_scope("CNN"):
    
        batch,_,_,_=tf.unstack(tf.shape(inputsq))
        qss=tf.reshape(inputsq,(batch*maxqsen,maxqword,emb))
        print(qss)
        conv1d=tf.layers.conv1d(qss,1,3)
        print(conv1d)
        b1,dim,_=tf.unstack(tf.shape(conv1d))
        print("ge")
        val=tf.reshape(conv1d,(b1,dim))
        print(val)
        topk=tf.nn.top_k(val,ns)
        print(topk)


        aa=tf.reshape(topk.values,(batch,maxqsen,ns))
        print(aa)



        rss=tf.reshape(inputsr,(batch*maxrsen,maxrword,emb))


        print(rss)
        conv1d=tf.layers.conv1d(rss,1,3)
        print(conv1d)
        b2,dim,_=tf.unstack(tf.shape(conv1d))

        val=tf.reshape(conv1d,(b2,dim))
        print(val)
        topk=tf.nn.top_k(val,ns)
        print(topk)

        bb=tf.reshape(topk.values,(batch,maxrsen,ns))
        print(bb)






Tensor("CNN/Reshape:0", shape=(?, 50, 50), dtype=float32)
Tensor("CNN/conv1d/BiasAdd:0", shape=(?, 48, 1), dtype=float32)
ge
Tensor("CNN/Reshape_1:0", shape=(?, ?), dtype=float32)
TopKV2(values=<tf.Tensor 'CNN/TopKV2:0' shape=(?, 20) dtype=float32>, indices=<tf.Tensor 'CNN/TopKV2:1' shape=(?, 20) dtype=int32>)
Tensor("CNN/Reshape_2:0", shape=(?, 5, 20), dtype=float32)
Tensor("CNN/Reshape_3:0", shape=(?, 100, 50), dtype=float32)
Tensor("CNN/conv1d_1/BiasAdd:0", shape=(?, 98, 1), dtype=float32)
Tensor("CNN/Reshape_4:0", shape=(?, ?), dtype=float32)
TopKV2(values=<tf.Tensor 'CNN/TopKV2_1:0' shape=(?, 20) dtype=float32>, indices=<tf.Tensor 'CNN/TopKV2_1:1' shape=(?, 20) dtype=int32>)
Tensor("CNN/Reshape_5:0", shape=(?, 25, 20), dtype=float32)


### Finding score of each questions and response pair by following equations

<img src="attention.PNG">

In [11]:
with g.as_default():
    
    with tf.variable_scope("NTN"):
        M= tf.Variable(tf.truncated_normal(shape=(ns,ns,a),dtype=dtypef))
        V=tf.Variable(tf.truncated_normal(shape=(2*ns,a),dtype=dtypef))
        U=tf.Variable(tf.truncated_normal(shape=(a,),dtype=dtypef))
        b=tf.Variable(tf.zeros(shape=(a,),dtype=dtypef))

        tmp1=tf.multiply(tf.reshape(tf.transpose(aa,(1,0,2)),(maxqsen,1,batch,ns)),tf.ones(shape=(maxqsen,maxrsen,batch,ns),dtype=dtypef))
        print(tmp1)
        tmp2=tf.multiply(tf.reshape(tf.transpose(bb,(1,0,2)),(1,maxrsen,batch,ns)),tf.ones(shape=(maxqsen,maxrsen,batch,ns),dtype=dtypef))
        print(tmp2)





        ft=tf.tensordot(tmp1,tf.reshape(M,(1,ns,ns,a)),axes=[[3],[1]])
        ft=tf.reshape(ft,(maxqsen,maxrsen,batch,ns,a))
        ft=tf.multiply(ft,tf.reshape(tmp2,(maxqsen,maxrsen,batch,ns,1)))
        ft=tf.reduce_sum(ft,axis=3)







        st=tf.reshape(tf.tensordot(tf.concat((tmp1,tmp2),axis=3),tf.reshape(V,(1,1,2*ns,a)),axes=[[3],[2]]),(maxqsen,maxrsen,batch,a))
        print(st)



        res=tf.reshape(tf.tensordot(tf.nn.tanh(tf.add(tf.add(ft,st),tf.reshape(b,(1,1,1,a)))),tf.reshape(U,(1,1,1,a)),axes=[[3],[3]]),(maxqsen*maxrsen,batch))


        print("res",res)




        ind=tf.cast(tf.mod(tf.argmax(res,axis=0),maxrsen),dtypei)
        print(ind)

        row_indices = tf.range(batch,dtype=dtypei)

        indices = tf.transpose([row_indices, ind])


        srclen=tf.cast(tf.gather_nd(src_seq_length,indices),dtypei)
        print(srclen,inputsr)



        responses=tf.cast(tf.gather_nd(inputsr, indices),dtypef)
    
    #sess=tf.Session()
    #sess.run(tf.global_variables_initializer())
    #indic,i=sess.run([ind,indices],{inputsq:qs[0:5],inputsr:rs[0:5],src_seq_length:rsqlen[0:5],trg_seq_length:fsqlen[0:5]})
    
    #print(indic,i)
    
    
    

Tensor("NTN/Mul:0", shape=(5, 25, ?, 20), dtype=float32)
Tensor("NTN/Mul_1:0", shape=(5, 25, ?, 20), dtype=float32)
Tensor("NTN/Reshape_6:0", shape=(5, 25, ?, 5), dtype=float32)
res Tensor("NTN/Reshape_9:0", shape=(125, ?), dtype=float32)
Tensor("NTN/Cast:0", shape=(?,), dtype=int32)
Tensor("NTN/GatherNd:0", shape=(?,), dtype=int32) Tensor("Placeholder_3:0", shape=(?, 25, 100, 50), dtype=float32)


## Seq to Seq model with Attention

In [12]:


src_emb_size=50
trg_emb_size=50

num_units=30
attention_a=5
#src_seq_len=30
#trg_seq_len=30
epochs=1
max_time=30

dtypei=tf.int32
dtypef=tf.float32



with g.as_default():

    labels=tf.placeholder(dtypef,[None,maxfword,trg_emb_size])

    #src_seq_length=tf.placeholder(shape=(None,), dtype=tf.int32)
    initialize=tf.placeholder(dtypef,[None,trg_emb_size+num_units])


    
    with tf.variable_scope("seq_to_seq"):

        
        Ua=tf.Variable(tf.truncated_normal(shape=(attention_a,),dtype=dtypef))
        ba=tf.Variable(tf.zeros(shape=(attention_a,),dtype=dtypef))
        Va=tf.Variable(tf.truncated_normal(shape=(2*num_units,attention_a),dtype=dtypef))
        Ma=tf.Variable(tf.truncated_normal(shape=(num_units,num_units,attention_a),dtype=dtypef))




    #encoder_word_embeddings = tf.Variable(tf.truncated_normal(shape=(src_vocab_size,src_emb_size)))
    #embedded_word_ids = tf.nn.embedding_lookup(encoder_word_embeddings, inputs)

        with tf.variable_scope("encoder"):


            encoder_lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(num_units)
            print(encoder_lstm_cell)

            encoder_lstm_net=tf.nn.dynamic_rnn(inputs=responses,cell=encoder_lstm_cell,dtype=dtypef,sequence_length=srclen)

            context_vector,encoder_final_state=encoder_lstm_net

            #context_vector=tf.multiply(tf.ones((1,max_time,num_units)),context_vector)
            print("context_vector",context_vector)

            lab=2




        #batchSize=tf.unstack(tf.shape(inputs))[0]
        #print(batchSize)
        #batchSize=tf.constant(batchSize)

        dense_w=tf.Variable(tf.truncated_normal(shape=(num_units,trg_emb_size)))
        dense_b=tf.Variable(tf.zeros(shape=(trg_emb_size,)))

        def loop_fn(time, cell_output, cell_state, loop_state):



              # == None for time == 0
            #print("time=",time)

            if cell_output is None:  # time == 0
                next_cell_state = encoder_final_state#cell.zero_state(batch_size, tf.float32)
                #inp=tf.zeros([batch_size, trg_emb_size+num_units], dtype=tf.float32)
                #print("if",next_cell_state)
                #print("if ",inputs)
                inp=initialize

            else:
                #print("a=",cell_output)
                next_cell_state = cell_state
                #print("else",next_cell_state)
              
                inp=tf.nn.sigmoid(tf.add(tf.matmul(cell_output,dense_w),dense_b))
                #print("temp",temp)


                print("cell_output",cell_output)

                print(batch_size)
                """"
                temp=tf.reshape(cell_output,(batch_size,1,num_units))

                temp=tf.multiply(context_vector,temp)

                alpha=tf.nn.softmax(tf.reshape(tf.reduce_sum(temp,axis=2),(batch_size,maxtime,1)),axis=1)
                print(alpha)
                """
                
                

                batch,_=tf.unstack(tf.shape(cell_output))
                
                with tf.variable_scope("attention"):

                    ft=tf.multiply(tf.tensordot(context_vector,Ma,axes=[[2],[0]]), tf.reshape(cell_output,(batch,1,num_units,1)))
                    ft=tf.reduce_sum(ft,axis=2)

                    print("ft",ft)

                    #print("ones",tf.concat((tf.multiply(tf.ones((1,max_time,num_units),dtype=tf.float32),tf.reshape(cell_output,(batch,1,num_units))),context_vector),axis=2))
                    st=tf.concat((tf.multiply(tf.ones((1,maxrword,num_units),dtype=tf.float32),tf.reshape(cell_output,(batch,1,num_units))),context_vector),axis=2)
                    print("st",st,tf.reshape(Va,(2*num_units,attention_a,1)))
                    print(tf.tensordot(st,tf.reshape(Va,(2*num_units,attention_a,1)),axes=[[2],[0]]))
                    #st=tf.concat((tf.multiply(tf.ones((1,max_time,num_units),dtype=tf.float32),tf.reshape(cell_output,(batch,1,num_units))),context_vector),axis=2)
                    st=tf.reshape(tf.tensordot(st,tf.reshape(Va,(2*num_units,attention_a,1)),axes=[[2],[0]]),(batch,maxrword,attention_a))

                    print("st",st)

                    tt=tf.reshape(ba,(1,1,attention_a))

                    print("tt",tt)

                    alpha=tf.nn.softmax(tf.reduce_sum(tf.multiply(tf.reshape(Ua,(1,1,attention_a)),tf.nn.tanh(tf.add(tf.add(ft,st),tt))),axis=2))
                    print("a",alpha)
                    alpha=tf.reshape(alpha,(batch,maxrword,1))

                    print("alpha",alpha)

                    temp=tf.multiply(alpha,context_vector)

                    ctx=tf.reduce_sum(temp,axis=1)

                inp=tf.concat([inp,ctx],axis=1)

                print("inp",inp)






            emit_output=cell_output
                #inp=tf.as.dese(cell_output,20,activation='softmax',name="dense"+(str)(time))
                #print("else ",inputs)





            print("emit",emit_output)
            elements_finished = (time >= trg_seq_length)
            print("elements_finished",elements_finished)
            finished = tf.reduce_all(elements_finished)
            #inp=tf.layers.dense(cell_output,20,activation='softmax')
            print(batch_size)
            next_input=inp
            """next_input = tf.cond(
              finished,
              lambda: initialize,
              lambda: inp)"""
            next_loop_state = None
            print(next_input)
            return (elements_finished, next_input, next_cell_state,
                  emit_output, next_loop_state)


        with tf.variable_scope("decoder"):
            decoder_lstm_cell =tf.contrib.rnn.LSTMCell(num_units)
            decoder_lstm_net=tf.nn.raw_rnn(decoder_lstm_cell, loop_fn)
            print("decoder",decoder_lstm_net)
            outputs_ta, final_state, _ = decoder_lstm_net
            outputs = outputs_ta.stack()
            print("outputs=",outputs)
            seq_len,batch,hidden_units=tf.unstack(tf.shape(outputs))
            new_out=tf.reshape(outputs,[seq_len*batch,hidden_units])
            temp=tf.nn.sigmoid(tf.add(tf.matmul(new_out,dense_w),dense_b))
            print("temp",temp)

            predicted=tf.transpose(tf.reshape(temp,[seq_len,batch,trg_emb_size]),[1,0,2])
            print("labels=",labels)
            print("predicted=",predicted)
            #cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=predicted))
            #print("losses=",tf.losses.softmax_cross_entropy(labels,predicted))
            shapepredicted=tf.shape(predicted)
            print("shape",tf.shape(labels))
            batch,seq_len,vocab=tf.unstack(tf.shape(predicted))
            lb=tf.slice(labels,[0,0,0],[batch,seq_len,vocab])


        print("labels=",lb)
        
    with tf.variable_scope("loss"):

        #cross_entropy=tf.losses.softmax_cross_entropy(lb,predicted)
        mean_square_loss=tf.losses.mean_squared_error(lb,predicted)
        print(mean_square_loss)
        train_step = tf.train.AdamOptimizer(1e-4).minimize(mean_square_loss)

        #correct_prediction = tf.equal(tf.argmax(predicted, 2), tf.argmax(lb, 2))
        #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #print("ac",accuracy)
       


    

context_vector Tensor("seq_to_seq/encoder/rnn/transpose_1:0", shape=(?, 100, 30), dtype=float32)
emit None
elements_finished Tensor("seq_to_seq/decoder/rnn/GreaterEqual:0", shape=(?,), dtype=bool)
1086
Tensor("Placeholder_5:0", shape=(?, 80), dtype=float32)
cell_output Tensor("seq_to_seq/decoder/rnn/while/lstm_cell/mul_2:0", shape=(?, 30), dtype=float32)
1086
ft Tensor("seq_to_seq/decoder/rnn/while/attention/Sum:0", shape=(?, 100, 5), dtype=float32)
st Tensor("seq_to_seq/decoder/rnn/while/attention/concat:0", shape=(?, 100, 60), dtype=float32) Tensor("seq_to_seq/decoder/rnn/while/attention/Reshape_2:0", shape=(60, 5, 1), dtype=float32)
Tensor("seq_to_seq/decoder/rnn/while/attention/Tensordot_1:0", shape=(?, 100, 5, 1), dtype=float32)
st Tensor("seq_to_seq/decoder/rnn/while/attention/Reshape_5:0", shape=(?, 100, 5), dtype=float32)
tt Tensor("seq_to_seq/decoder/rnn/while/attention/Reshape_6:0", shape=(1, 1, 5), dtype=float32)
a Tensor("seq_to_seq/decoder/rnn/while/attention/Softmax:0", s

## Writing summary on tensorboard

In [14]:

writer=tf.summary.FileWriter('models/',g)


### Training and Testing 

In [13]:
pred=""
with g.as_default():
    with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            bt=543
            epochs=100
            for k in range(epochs):
                print("epoch ",k)
                sess.run(train_step,{initialize:np.zeros((bt,trg_emb_size+num_units)),inputsq:qs[0:bt],inputsr:rs[0:bt],labels:fs[0:bt],src_seq_length:rsqlen[0:bt],trg_seq_length:fsqlen[0:bt]})
                
                sess.run(train_step,{initialize:np.zeros((bt,trg_emb_size+num_units)),inputsq:qs[bt:],inputsr:rs[bt:],labels:fs[bt:],src_seq_length:rsqlen[bt:],trg_seq_length:fsqlen[bt:]})
                m=sess.run(mean_square_loss,{initialize:np.ones((bt,trg_emb_size+num_units)),inputsq:qs[0:bt],inputsr:rs[0:bt],labels:fs[0:bt],src_seq_length:rsqlen[0:bt],trg_seq_length:fsqlen[0:bt]})
                print("mean square loss for batch 1",m)
                sess.run(mean_square_loss,{initialize:np.ones((bt,trg_emb_size+num_units)),inputsq:qs[bt:],inputsr:rs[bt:],labels:fs[bt:],src_seq_length:rsqlen[bt:],trg_seq_length:fsqlen[bt:]})
                print("mean square loss for batch 2",m)
                
                
                
            pred=sess.run(predicted,{initialize:np.ones((bt,trg_emb_size+num_units)),inputsq:qs[0:bt],inputsr:rs[0:bt],labels:fs[0:bt],src_seq_length:rsqlen[0:bt],trg_seq_length:fsqlen[0:bt]})
            
            #print(finals.c.shape)



epoch  0
mean square loss for batch 1 0.3858786
mean square loss for batch 2 0.3858786
epoch  1
mean square loss for batch 1 0.38195533
mean square loss for batch 2 0.38195533
epoch  2
mean square loss for batch 1 0.37807828
mean square loss for batch 2 0.37807828
epoch  3
mean square loss for batch 1 0.37425688
mean square loss for batch 2 0.37425688
epoch  4
mean square loss for batch 1 0.3705012
mean square loss for batch 2 0.3705012
epoch  5
mean square loss for batch 1 0.36682117
mean square loss for batch 2 0.36682117
epoch  6
mean square loss for batch 1 0.363226
mean square loss for batch 2 0.363226
epoch  7
mean square loss for batch 1 0.35972324
mean square loss for batch 2 0.35972324
epoch  8
mean square loss for batch 1 0.35631835
mean square loss for batch 2 0.35631835
epoch  9
mean square loss for batch 1 0.35301453
mean square loss for batch 2 0.35301453
epoch  10
mean square loss for batch 1 0.34981325
mean square loss for batch 2 0.34981325
epoch  11
mean square loss f

mean square loss for batch 1 0.24261987
mean square loss for batch 2 0.24261987
epoch  93
mean square loss for batch 1 0.24193858
mean square loss for batch 2 0.24193858
epoch  94
mean square loss for batch 1 0.24125911
mean square loss for batch 2 0.24125911
epoch  95
mean square loss for batch 1 0.24058136
mean square loss for batch 2 0.24058136
epoch  96
mean square loss for batch 1 0.23990518
mean square loss for batch 2 0.23990518
epoch  97
mean square loss for batch 1 0.23923053
mean square loss for batch 2 0.23923053
epoch  98
mean square loss for batch 1 0.23855737
mean square loss for batch 2 0.23855737
epoch  99
mean square loss for batch 1 0.23788571
mean square loss for batch 2 0.23788571


## Finding euclidean distance between predicted embedding and word to vec vocabulary

In [15]:
for i in mispell:
    word_vector.pop(i,None)
    
keys=np.array(list(word_vector.keys()))
    
disp=[]
disa=[]
for i in word_vector.keys():
    #print(i)
   

    temp=np.array(word_vector[i]).reshape(1,1,50)

    disp.append(np.linalg.norm(temp-pred[0:50],ord=2,axis=2))
    disa.append(np.linalg.norm(temp-fs[0:50],ord=2,axis=2))
disp=np.array(disp)
disa=np.array(disa)
dp=np.argmin(disp,axis=0)
da=np.argmin(disa,axis=0)
  

## Predicted Sentence

In [16]:
keys[dp[16]]

array(['importantly', 'importantly', 'importantly', 'importantly',
       'luckily', 'luckily', 'luckily', 'luckily', 'luckily', 'luckily',
       'luckily', 'luckily', 'luckily', 'luckily', 'luckily', 'luckily',
       'luckily', 'luckily', 'luckily', 'luckily', 'luckily', 'luckily',
       'luckily', 'luckily', 'luckily', 'luckily', 'luckily', 'luckily',
       'luckily', 'luckily', 'luckily', 'luckily', 'luckily', 'luckily',
       'luckily', 'luckily', 'luckily', 'luckily', 'luckily', 'luckily',
       'luckily', 'luckily', 'luckily', 'luckily', 'luckily', 'luckily',
       'luckily'], dtype='<U17')

## Target Sentence

In [17]:
keys[da[16]]

array(['why', 'should', 'i', 'hire', 'you', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating', 'emulating', 'emulating',
       'emulating', 'emulating', 'emulating'], dtype='<U17')